In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import xgboost as xgb


# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
USE DATABASE ENPAL_ITALY;
USE SCHEMA RAW;


In [ ]:

select * from lead_time

In [ ]:
  SELECT 
            r.id AS requestid,
            zipregion,
            r.evaluationtime,
            r.desiredinstallationend,
            electricitybill,
            heatingbill,
            channel__campaign,
            l.channel,
            a.grosscontractsigned,
           selfipaimportedat
        FROM request r
        LEFT JOIN lead l 
            ON l.id = r.leadid
        LEFT JOIN airtable_contracts a 
            ON a.requestid = r.id

In [ ]:
WITH all_events AS (
  SELECT 
    createdat, 
    deletedat, 
    requestid, 
    tsstart, 
    type, 
    is_passed, 
    leadid
  FROM calendar_event
  WHERE type IN ('VD', 'VDFIELD')
),
latest_activity AS (
  SELECT 
    requestid,
    MAX(createdat) AS latest_date
  FROM all_events
  GROUP BY requestid
),
airtable_req AS (
  SELECT DISTINCT requestid
  FROM airtable_contracts
  WHERE requestid IS NOT NULL
),
-- Group A: requests with NO recent activity in the last 3 months
no_recent_activity AS (
  SELECT 
    e.*
  FROM all_events e
  JOIN latest_activity la ON e.requestid = la.requestid
  WHERE la.latest_date < CURRENT_DATE - INTERVAL '3 months'
),
-- Group B: requests that have a record in airtable_contracts
with_contract AS (
  SELECT 
    e.*
  FROM all_events e
  JOIN airtable_req a ON e.requestid = a.requestid
)
-- Combine and remove duplicates
SELECT *
FROM no_recent_activity
UNION  -- not UNION ALL!
SELECT *
FROM with_contract

In [ ]:
 WITH c AS (
  SELECT 
    "Date",
    "telephoneNumber",
    "Wrap-up",
    RIGHT(REGEXP_REPLACE("telephoneNumber", '[^0-9]', ''), 10) AS cleaned_phone
  FROM genesys_db
  WHERE "Queue" = 'SELLER'
    AND "Wrap-up" IN ('FU_FATTO', 'FU_SENZA RISPOSTA')
),
s AS (
  SELECT 
    phonenumber, 
    id,
    RIGHT(REGEXP_REPLACE(phonenumber, '[^0-9]', ''), 10) AS cleaned_phone
  FROM lead
),
joined_data AS (
  SELECT 
    c."Date", 
    c."Wrap-up", 
    r.id AS requestid
  FROM c
  JOIN s ON c.cleaned_phone = s.cleaned_phone
  JOIN request r ON r.leadid = s.id
  WHERE s.cleaned_phone IS NOT NULL
    AND s.id IS NOT NULL
),
latest_activity AS (
  SELECT 
    requestid,
    MAX("Date") AS latest_date
  FROM joined_data
  GROUP BY requestid
),
airtable_req AS (
  SELECT DISTINCT requestid
  FROM airtable_contracts
  WHERE requestid IS NOT NULL
),
-- Group A: No recent activity in last 3 months
no_recent_activity AS (
  SELECT 
    j."Date",
    j."Wrap-up",
    j.requestid,
    'NO_RECENT_ACTIVITY' AS group_type
  FROM joined_data j
  JOIN latest_activity la ON j.requestid = la.requestid
  WHERE la.latest_date < CURRENT_DATE - INTERVAL '3 months'
),
-- Group B: Has a signed contract
with_contract AS (
  SELECT 
    j."Date",
    j."Wrap-up",
    j.requestid,
    'WITH_CONTRACT' AS group_type
  FROM joined_data j
  JOIN airtable_req a ON j.requestid = a.requestid
)
-- Union both groups
SELECT   "Date",
    "Wrap-up",
    requestid,
FROM no_recent_activity
UNION 
SELECT  "Date",
    "Wrap-up",
    requestid,
FROM with_contract;

In [ ]:
select distinct id as requestid, leadid from request

In [ ]:
WITH first_dispatcher AS (
    SELECT 
        leadid,
        createdat AS first_dispatcher_createdat
    FROM dispatcher_answers
    WHERE outcome = 'VDTOPLAN'
    QUALIFY ROW_NUMBER() OVER (
        PARTITION BY leadid
        ORDER BY createdat ASC
    ) = 1
),

first_calendar AS (
    SELECT
        leadid,
        tsstart AS first_calendar_createdat
    FROM calendar_event
    WHERE type IN ('VD', 'VDFIELD')  -- 👈 replace with your actual list of types
      AND deletedat IS NULL
    QUALIFY ROW_NUMBER() OVER (
        PARTITION BY leadid
        ORDER BY createdat ASC
    ) = 1
)

SELECT
    d.leadid,
    ROUND(
        DATEDIFF('second', d.first_dispatcher_createdat, c.first_calendar_createdat) / 3600.0,
        2
    ) AS bc_to_sc_hour
FROM first_dispatcher d
LEFT JOIN first_calendar c 
    ON d.leadid = c.leadid
WHERE c.first_calendar_createdat > d.first_dispatcher_createdat;


In [ ]:
#leadtime added bc to sc 
leadtime = cell2.to_pandas()
lead_bc_to_df = cell19.to_pandas()

In [ ]:
lead_bc_to_df.columns = lead_bc_to_df.columns.str.lower()

In [ ]:
leadtime.info()

In [ ]:
# Then, we can use the python name to turn cell2 into a Pandas dataframe

client_info = cell4.to_pandas()
sc_df = cell5.to_pandas()
fu_df = cell6.to_pandas()
requestid_leadid_pairs = cell7.to_pandas()





In [ ]:
client_info.columns = client_info.columns.str.lower()
sc_df.columns = sc_df.columns.str.lower()
fu_df.columns = fu_df.columns.str.lower()
requestid_leadid_pairs.columns = requestid_leadid_pairs.columns.str.lower()

In [ ]:
def customer_behaviour_leadtime(df):
   
        
        # Create a copy to avoid modifying original data
        df_features = df.copy()
        
        # Convert date columns to datetime
        date_columns = ['LEAD_CREATION_DATE', 'SC1_SCHEDULED', 'SC1_APPOINTMENT', 
                       'LAST_FU', 'NEXT_FU', 'CONTRACT_SIGNED', 'WITHDRAWAL', 
                       'IPA_SCHEDULED', 'IPA_DAY']
        
        for col in date_columns:
            if col in df_features.columns:
                df_features[col] = pd.to_datetime(df_features[col], errors='coerce')
        
        # BC date columns
        bc_date_cols = [f'BC_{i}' for i in range(1, 19)]
        for col in bc_date_cols:
            if col in df_features.columns:
                df_features[col] = pd.to_datetime(df_features[col], errors='coerce')
        
        # BC outcome columns
        bc_outcome_cols = [f'BC_{i}_OUTCOME' for i in range(1, 19)]
        
        
        
        # 1. BOOKING CALL EFFICIENCY FEATURES
        # Count total BC attempts (non-null BC dates)
        bc_dates = df_features[bc_date_cols].copy()
        df_features['total_bc_attempts'] = bc_dates.notna().sum(axis=1)
        
        # Count total BC outcomes (non-null outcomes)
        bc_outcomes = df_features[bc_outcome_cols].copy()
        df_features['total_bc_outcomes'] = bc_outcomes.notna().sum(axis=1)
        
        # First and last BC attempt dates
        df_features['first_bc_date'] = bc_dates.min(axis=1)
        df_features['last_bc_date'] = bc_dates.max(axis=1)
        
   
        
        # 2. TEMPORAL FEATURES
        # Days from lead creation to first BC
        df_features['lead_to_first_bc_days'] = (
            df_features['first_bc_date'] - df_features['LEAD_CREATION_DATE']
        ).dt.days
        
        # Days from first BC to last BC (booking call duration)
        df_features['bc_duration_days'] = (
            df_features['last_bc_date'] - df_features['first_bc_date']
        ).dt.days
        
        # Days from lead creation to SC1 scheduled
        df_features['lead_to_sc1_days'] = (
            df_features['SC1_SCHEDULED'] - df_features['LEAD_CREATION_DATE']
        ).dt.days
        
        # Days from SC1 scheduled to appointment
        df_features['sc1_schedule_to_appointment_days'] = (
            df_features['SC1_APPOINTMENT'] - df_features['SC1_SCHEDULED']
        ).dt.days
        
        # BC call frequency (attempts per day during BC period)
        df_features['bc_frequency'] = np.where(
            df_features['bc_duration_days'] > 0,
            df_features['total_bc_attempts'] / (df_features['bc_duration_days'] + 1),
            df_features['total_bc_attempts']  # If same day, frequency = total attempts
        )
        
        
        # 3. ENGAGEMENT QUALITY FEATURES
        # Define outcome categories
        positive_outcomes = ['VDTOPLAN', 'REACHED', 'CONFIRMED', 'INTERESTED']
        negative_outcomes = ['LOST', 'NOTINTERESTED', 'UNQUALIFIED']
        neutral_outcomes = ['TOBERECALLED', 'RESCHEDULE']
        unreachable_outcomes = ['NOTREACHED', 'NOANSWER', 'VOICEMAIL']
        noshow_outcomes = ['NOSHOW', 'NOSHOWNOTREACHED', 'NOSHOWLOST', 'NOSHOWVDTOPLAN', 'NOSHOWTOBERECALLED']
        
        # Count outcomes by category
        def count_outcome_category(row, category_list):
            count = 0
            for col in bc_outcome_cols:
                if col in row.index and pd.notna(row[col]):
                    if any(cat.upper() in str(row[col]).upper() for cat in category_list):
                        count += 1
            return count
        
        df_features['positive_outcomes_count'] = df_features.apply(
            lambda row: count_outcome_category(row, positive_outcomes), axis=1
        )
        
        df_features['negative_outcomes_count'] = df_features.apply(
            lambda row: count_outcome_category(row, negative_outcomes), axis=1
        )
        
        df_features['noshow_outcomes_count'] = df_features.apply(
            lambda row: count_outcome_category(row, noshow_outcomes), axis=1
        )
        
        df_features['unreachable_outcomes_count'] = df_features.apply(
            lambda row: count_outcome_category(row, unreachable_outcomes), axis=1
        )
        
        # Calculate ratios (avoid division by zero)
        total_outcomes = df_features['total_bc_outcomes']
        df_features['positive_outcome_ratio'] = np.where(
            total_outcomes > 0, 
            df_features['positive_outcomes_count'] / total_outcomes, 
            0
        )
        
        df_features['negative_outcome_ratio'] = np.where(
            total_outcomes > 0,
            df_features['negative_outcomes_count'] / total_outcomes,
            0
        )
        
        df_features['noshow_outcome_ratio'] = np.where(
            total_outcomes > 0,
            df_features['noshow_outcomes_count'] / total_outcomes,
            0
        )
        
        df_features['reachability_score'] = np.where(
            total_outcomes > 0,
            1 - (df_features['unreachable_outcomes_count'] / total_outcomes),
            0
        )
        
        
        
        # 4. BEHAVIORAL PATTERN FEATURES
        # Last BC outcome (most recent attempt result)
        def get_last_bc_outcome(row):
            for i in range(18, 0, -1):  # Check from BC_18 down to BC_1
                col = f'BC_{i}_OUTCOME'
                if col in row.index and pd.notna(row[col]):
                    return row[col]
            return 'NO_OUTCOME'
        
        df_features['last_bc_outcome'] = df_features.apply(get_last_bc_outcome, axis=1)
        
        # First BC outcome
        def get_first_bc_outcome(row):
            for i in range(1, 19):  # Check from BC_1 to BC_18
                col = f'BC_{i}_OUTCOME'
                if col in row.index and pd.notna(row[col]):
                    return row[col]
            return 'NO_OUTCOME'
        
        df_features['first_bc_outcome'] = df_features.apply(get_first_bc_outcome, axis=1)
        
        # Outcome improvement/deterioration pattern
        def outcome_trend(row):
            outcomes = []
            for i in range(1, 19):
                col = f'BC_{i}_OUTCOME'
                if col in row.index and pd.notna(row[col]):
                    # Score outcomes (higher = better)
                    if any(pos.upper() in str(row[col]).upper() for pos in positive_outcomes):
                        outcomes.append(3)
                    elif any(neu.upper() in str(row[col]).upper() for neu in neutral_outcomes):
                        outcomes.append(2)
                    elif any(unr.upper() in str(row[col]).upper() for unr in unreachable_outcomes):
                        outcomes.append(1)
                    else:
                        outcomes.append(0)  # negative outcomes
            
            if len(outcomes) < 2:
                return 0  # No trend
            
            # Calculate trend (positive = improving, negative = deteriorating)
            return outcomes[-1] - outcomes[0]
        
        df_features['outcome_trend'] = df_features.apply(outcome_trend, axis=1)
        
        # Persistence score (attempts after first negative outcome)
        def persistence_score(row):
            first_negative_pos = None
            total_after_negative = 0
            
            for i in range(1, 19):
                col = f'BC_{i}_OUTCOME'
                if col in row.index and pd.notna(row[col]):
                    if first_negative_pos is None and any(neg.upper() in str(row[col]).upper() for neg in negative_outcomes):
                        first_negative_pos = i
                    elif first_negative_pos is not None:
                        total_after_negative += 1
            
            return total_after_negative
        
        df_features['persistence_after_negative'] = df_features.apply(persistence_score, axis=1)
        
       
        
        # 5. CONVERSION-RELATED FEATURES
        # Binary target
        df_features['converted'] = df_features['CONTRACT_SIGNED'].notna().astype(int)
        
     
        # Show-up for SC1 appointment
        df_features['showed_up_sc1'] = df_features['SC1_APPOINTMENT'].notna().astype(int)
        
    
        
        # 6. COMPOSITE SCORES
        # Overall engagement score (weighted combination)
        df_features['engagement_score'] = (
            0.3 * df_features['positive_outcome_ratio'] + 
            0.2 * df_features['reachability_score'] + 
            0.2 * (1 - df_features['negative_outcome_ratio']) +
            0.1 * (1 - df_features['noshow_outcome_ratio']) +
            0.1 * np.clip(df_features['bc_frequency'], 0, 5) / 5 +  # Cap frequency at 5
            0.1 * np.clip(df_features['outcome_trend'], -3, 3) / 6  # Normalize trend
        )
        
        # Efficiency score (quick to schedule with few attempts)
        df_features['efficiency_score'] = np.where(
            df_features['total_bc_attempts'] > 0,
            1 / (1 + df_features['total_bc_attempts']),  # Inverse of attempts
            0
        )
        
     
        
        # Fill NaN values in temporal features
        temporal_features = [
            'lead_to_first_bc_days', 'bc_duration_days', 'lead_to_sc1_days',
            'sc1_schedule_to_appointment_days', 'bc_frequency'
        ]
        
        for feature in temporal_features:
            if feature in df_features.columns:
                df_features[feature] = df_features[feature].fillna(0)
        
      
        
        # Return only the engineered features (remove original columns for cleaner output)
        feature_columns = [
            'ID', 'converted',  # Keep ID and target
            # Booking call efficiency
            'total_bc_attempts', 'total_bc_outcomes',
            # Temporal features
            'lead_to_first_bc_days', 'bc_duration_days', 'lead_to_sc1_days',
            'sc1_schedule_to_appointment_days', 'bc_frequency',
            # Engagement quality
            'positive_outcomes_count', 'negative_outcomes_count', 'noshow_outcomes_count',
            'positive_outcome_ratio', 'negative_outcome_ratio', 'noshow_outcome_ratio', 'reachability_score',
            # Behavioral patterns
            'last_bc_outcome', 'first_bc_outcome', 'outcome_trend', 'persistence_after_negative',
            # Conversion related
            'showed_up_sc1',
            # Composite scores
            'engagement_score', 'efficiency_score'
        ]
        
        df_features = df_features[feature_columns]
    
        #encoding categorical variables
    
        df_model = df_features.copy()
    
        # Label encode the categorical feature
        le = LabelEncoder()
        df_model['last_bc_outcome_encoded'] = le.fit_transform(df_model['last_bc_outcome'])
    
        # Drop the original categorical column
        df_model = df_model.drop('last_bc_outcome', axis=1)
    
        # Encode first_bc_outcome
        le_first = LabelEncoder()
        df_model['first_bc_outcome_encoded'] = le_first.fit_transform(df_model['first_bc_outcome'])
    
        # Drop the original categorical column
        df_model = df_model.drop('first_bc_outcome', axis=1)
    
        return df_model
            

In [ ]:
def info_client_request(df3):
            
        def get_aggregated_value(row):
            # Check Youtube first
            if row.get('Youtube') == 1:
                return "Youtube"
            
            # Check specific CHANNEL__CAMPAIGN values
            channel_campaign = row.get('channel_campaign')
            if channel_campaign == "d2d":
                return "d2d"
            elif channel_campaign == "form_classico":
                return "Form_FB"
            elif channel_campaign == "ranger":
                return "ranger"
            elif channel_campaign == "referral_link":
                return "Referral"
            elif channel_campaign == "referral_manual":
                return "Referral"
            
            # Check Channel column (case insensitive)
            channel = row.get('channel')
            if channel and pd.notna(channel):
                channel_lower = channel.lower()
                
                if channel_lower == "affiliation":
                    return "Affiliation"
                elif channel_lower == "outbrain":
                    return "Outbrain"
                elif channel_lower in ["taboola", "taboola_it"]:
                    return "Taboola"
                elif channel_lower in ["facebook", "fb", "ig", "instagram", "facebook_marketplace"]:
                    return "Meta"
                elif channel_lower in ["search ads", "searchads", "google", "google ads"]:
                    return "Google"
                elif channel_lower in ["organic_brand", "organic", "direct"]:
                    return "Organic"
                elif channel_lower == "seo":
                    return "SEO"
                elif channel_lower == "criteo":
                    return "Criteo"
                elif channel_lower == "mediago":
                    return "MediaGo"
                elif channel_lower == "tiktok":
                    return "Tik Tok"
            
            return "Other"
        
        df3['aggregated'] = df3.apply(get_aggregated_value, axis=1)
        df3.drop('channel', axis=1, inplace=True)
        df3.drop('channel__campaign', axis=1, inplace=True)
        df3['selfipa_done'] = df3['selfipaimportedat'].notnull().astype(int)
        df3.drop('selfipaimportedat', axis=1, inplace=True) 
        return df3

In [ ]:
def transform_sc(df_fu, df_sc):

       # transforming sc and fu records
         # start with fu records
        
        df_fu['grouped_type'] = 'FU'
        df_fu.columns = df_fu.columns.str.lower()

        # net event calculation
        df_fu['is_net_event'] = 0
        df_fu.loc[df_fu['wrap-up'] == 'FU_FATTO', 'is_net_event']= 1 

        #drop wrap up column after calculation
        df_fu = df_fu.drop(columns = 'wrap-up')

        # fix date colum
        df_fu['date'] = pd.to_datetime(df_fu['date'])
    

        # transform sc
        #assign type
       
        df_sc.columns = df_sc.columns.str.lower()
        df_sc['grouped_type'] = 'SC'

        #net event calculation
        df_sc['is_net_event'] = 0
        df_sc.loc[df_sc['deletedat'].isna() & df_sc['is_passed'], 'is_net_event'] = 1
        #delete unneeded columns
        df_sc = df_sc.drop(columns = ['is_passed', 'deletedat', 'createdat'])

        #rename tsstart to date to macth the rest of the df
        df_sc.rename(columns ={
        'tsstart': 'date'
        }, inplace=True)

        #fix datetime
        df_sc['date']= pd.to_datetime(df_sc['date'])

        #put fu and sc together 
        df = pd.concat([df_sc, df_fu], ignore_index=True)

        #set is net to boolean for calculations
        df['is_net_event'] = df['is_net_event'].astype(bool)

       
                # Gross counts (all events)
        gross_counts = df.groupby(['requestid', 'grouped_type']).size().unstack(fill_value=0)
        gross_counts.columns = [f'gross_{col}' for col in gross_counts.columns]

        # Net counts (only events that actually happened)
        net_counts = df[df['is_net_event']].groupby(['requestid', 'grouped_type']).size().unstack(fill_value=0)
        net_counts.columns = [f'net_{col}' for col in net_counts.columns]

        # Combine gross and net counts_
        counts_df = gross_counts.join(net_counts, how='outer').fillna(0)

        # at this point requestid is the index. make it explicit
        counts_df = counts_df.reset_index()
    
        return counts_df

In [ ]:
def join_dfs(leadtime, sc_df, client_info, requestid_leadid_pairs):
             #merge client info and sc records 
        final_df = sc_df.merge(client_info, on='requestid', how='left')

        leadtime.columns = leadtime.columns.str.lower()

    
        # dont know why but lets impute missing values here 
        final_df['electricitybill'] = final_df['electricitybill'].replace([0, 1, -1], pd.NA)
        final_df['heatingbill'] = final_df['heatingbill'].replace([0, 1, -1], pd.NA)
    
        # also encoding why not
    
        # mark the missingness inside the row;
        missing_cols = ['zipregion', 'evaluationtime', 'desiredinstallationend', 'electricitybill', 'heatingbill', 'aggregated']
    
        for col in missing_cols:
            final_df[f'{col}_missing'] = final_df[col].isnull().astype(int)
    
        # desiredinstallationend
        type_mapping = {
            # Original Italian values
            '3-4mesi': 'three_to_four_months', 
            '5+mesi': 'more_than_5_months',
            '1-2mesi': 'one_to_two_months',
            'Non lo so': 'dont_know',
            'short': np.nan,
            # Already mapped values (keep as-is)
            'dont_know': 'dont_know',
            'three_to_four_months': 'three_to_four_months',
            'one_to_two_months': 'one_to_two_months', 
            'more_than_5_months': 'more_than_5_months',
            # Handle string 'nan'
            'nan': np.nan
        }
    
        final_df['desiredinstallationend1'] = final_df['desiredinstallationend'].map(type_mapping)
    
            # evaluationtime
        type_mapping = {
            # Original Italian values
            '3-6 mesi': np.nan,
            '<3 mesi': np.nan,
            '>6 mesi': np.nan,
            # Already mapped English values
            'less_than_three_months': np.nan,
            'more_than_six_months': np.nan,
            # Other values that appear in your data
            'understand_need': 'understand_need',  # or map to np.nan if you don't want these
            'understand_purchase': 'understand_purchase',  # or map to np.nan if you don't want these
            'evaluation': 'evaluation',  # or map to np.nan if you don't want these
            'curious': 'curious',  # or map to np.nan if you don't want these
            # Handle string 'nan'
            'nan': np.nan
        }
    
        # Create grouped_type column
        final_df['evaluationtime1'] = final_df['evaluationtime'].map(type_mapping)
    
        # Replace the old columns directly
        final_df['desiredinstallationend'] = final_df['desiredinstallationend1']
        final_df['evaluationtime'] = final_df['evaluationtime1']
    
        # Drop the temporary columns
        final_df = final_df.drop(['desiredinstallationend1', 'evaluationtime1'], axis=1)
    
        # handle nan before encoding
        final_df['desiredinstallationend'] = final_df['desiredinstallationend'].fillna('missing')
        final_df['evaluationtime'] = final_df['evaluationtime'].fillna('missing')
    
        
    
        # for marketing and region. nb this will need to be analyzed and updated every once in a while
        aggregated_groups = {
        'High': ['Affiliation', 'Taboola'],           # >25% conversion
        'Medium': ['Google', 'Criteo', 'Organic'],    # 15-20% conversion  
        'Low': ['Outbrain', 'Meta', 'Other', 'MediaGo', 'Tik Tok']  # <12% conversion
    }
        def group_aggregated(value):
            if value == 'SEO':  # Handle the unreliable outlier
                return 'Medium'  # Conservative assignment
            
            for group, channels in aggregated_groups.items():
                if value in channels:
                    return group
            return 'Low'  # fallback
    
    # Apply grouping
        final_df['mktg_grouped'] = final_df['aggregated'].apply(group_aggregated)
    
        # Group regions by performance
        def group_regions(region):
            # Top performers with good sample sizes
            if region in ['Trentino-Alto Adige', 'Friuli-Venezia Giulia', 'Liguria']:
                return 'High_Performer'  # 18-24% conversion
            
            # Large regions with solid performance  
            elif region in ['Lombardia', 'Piemonte', 'Veneto', 'Toscana', 'Emilia-Romagna']:
                return 'Large_Solid'     # 11-14% conversion
            
            # Smaller regions with decent samples
            elif region in ['Umbria', 'Lazio', 'Marche']:
                return 'Medium'          # 10-13% conversion
            
            # Lower performing regions
            else:
                return 'Lower'   
    
        final_df['region_grouped'] = final_df['zipregion'].apply(group_regions)
        final_df = pd.get_dummies(final_df, columns=['mktg_grouped', 'region_grouped'], prefix=['mktg', 'region'])
    
   
    
        # also add leadid to the final df for merging with behaviour data
        # probably there is a cleaner way to do this
    
        # Suppose the column you want from df1 is called "col_from_df1"
        final_df = final_df.merge(
            requestid_leadid_pairs,  # keep only requestid + desired column
            on='requestid',                      # join key
            how='left'                           # keep all rows from df2
        )
        #drop columns we encoded
        columns_to_drop = [ 'zipregion', 'evaluationtime', 'desiredinstallationend', 'aggregated']
        final_df = final_df.drop(columns=columns_to_drop)
    
        #imputing
        # Fix electricitybill missing values
        final_df['electricitybill'] = final_df['electricitybill'].fillna(final_df['electricitybill'].median())
        final_df['heatingbill'] = final_df['heatingbill'].fillna(final_df['heatingbill'].median())
    
        #scaling 
        continuous_cols= [
           
            'electricitybill', 
            'heatingbill'
        ]
    
        # Scale only the continuous features
        scaler = StandardScaler()
        final_df[continuous_cols] = scaler.fit_transform(final_df[continuous_cols])
    
    
        #finally merge with leadtime
    
        merged_df = final_df.merge(leadtime, 
                              left_on='leadid', 
                              right_on='id', 
                              how='left')
        
        return merged_df

In [ ]:

leadtime = customer_behaviour_leadtime(leadtime)
leadtime.columns = leadtime.columns.str.lower()
leadtime= pd.merge(leadtime, lead_bc_to_df, left_on='id', right_on='leadid', how='left').drop(columns='leadid')
sc_df = transform_sc(fu_df, sc_df)
client_info = info_client_request(client_info)
final_df = join_dfs(leadtime, sc_df, client_info, requestid_leadid_pairs)
final_df.columns = final_df.columns.str.lower()
final_df['grosscontractsigned'] = final_df['grosscontractsigned'].fillna(0)


In [ ]:
final_df.info()

Analysis of final df

Data analysis and feature analysis

In [ ]:
df = final_df.copy()

In [ ]:
df['bc_to_sc_hour'].var(ddof=0)


In [ ]:
df.describe(include='all')
df.isna().sum().sort_values(ascending=False)

In [ ]:
# variance and ditribution
from sklearn.feature_selection import VarianceThreshold

selector = VarianceThreshold(threshold=0.01)
selector.fit(df.select_dtypes(include=['int64', 'float64']))
low_var_cols = df.select_dtypes(include=['int64', 'float64']).columns[~selector.get_support()]
print(low_var_cols)


In [ ]:
import matplotlib.pyplot as plt
df['gross_fu'].hist(bins=50)
print(plt.show())


In [ ]:
corr = df.corr(numeric_only=True)
corr['converted'].sort_values(ascending=False)


In [ ]:
import seaborn as sns
sns.heatmap(corr, cmap='coolwarm', center=0)
plt.show()


In [ ]:
from xgboost import XGBClassifier
model = XGBClassifier()
X = df.drop(columns=['converted', 'requestid', 'id', 'leadid', 'grosscontractsigned'])
y = df['grosscontractsigned']
model.fit(X, y)
# Create feature importance DataFrame
importance = pd.DataFrame({
    'Feature': X.columns,
    'Importance': model.feature_importances_
}).sort_values(by='Importance', ascending=False)

# Plot
plt.figure(figsize=(10, 8))
sns.barplot(data=importance.head(20), x='Importance', y='Feature', palette='viridis')
plt.title('Top 20 Feature Importances (XGBoost)', fontsize=14)
plt.xlabel('Importance Score')
plt.ylabel('Feature')
plt.tight_layout()
plt.show()


In [ ]:
#take x because its dropped id and stuff 
variances = X.var(numeric_only=True)
variances.sort_values(ascending=False).head(15)


In [ ]:
variances.sort_values().head(15)


In [ ]:
df[['lead_to_sc1_days','bc_duration_days','lead_to_first_bc_days']].hist(bins=50, figsize=(10,6))


In [ ]:

ids = final_df["requestid"]
df = final_df.drop(columns=["requestid","id","converted","leadid"])
y_sc = (df['net_sc'] >= 1).astype(int)  # Stage 1: Did they reach SC?
y_sign = df['grosscontractsigned']       # Stage 2: Did they sign?

# Drop target from features
X = df.drop('grosscontractsigned', axis=1)

# Split with stratification on y_sc (since that's our first model)
X_train, X_test, y_train_sc, y_test_sc, y_train_sign, y_test_sign = train_test_split(
X, y_sc, y_sign, test_size=0.2, stratify=y_sc, random_state=42
)
X_layer1 = X.drop(columns=['gross_fu','gross_sc','net_fu','net_sc',
                            'selfipa_done',
                           'showed_up_sc1','sc1_schedule_to_appointment_days','lead_to_sc1_days'])
X_layer2 = X.copy()

In [ ]:
# TRAIN MODEL_SC (stage 1)

# Calculate class weights on TRAINING data only
scale_pos_weight_sc = (len(y_train_sc) - y_train_sc.sum()) / y_train_sc.sum()

model_sc = xgb.XGBClassifier(
    n_estimators=100, max_depth=4, learning_rate=0.05,
    subsample=0.8, colsample_bytree=0.8,
    scale_pos_weight=scale_pos_weight_sc, random_state=42,
    use_label_encoder=False, eval_metric="logloss"
)

# Fit on training data with layer1 features
model_sc.fit(X_layer1.loc[X_train.index], y_train_sc)

# TRAIN MODEL_SIGN (stage 2)

# Mask: only train on samples that reached SC stage
mask_train_sc = (df.loc[X_train.index, 'net_sc'] >= 1)

# Calculate class weights on the subset that reached SC
scale_pos_weight_sign = (mask_train_sc.sum() - y_train_sign[mask_train_sc].sum()) / y_train_sign[mask_train_sc].sum()

model_sign = xgb.XGBClassifier(
    n_estimators=100, max_depth=4, learning_rate=0.05,
    subsample=0.8, colsample_bytree=0.8,
    scale_pos_weight=scale_pos_weight_sign, random_state=42,
    use_label_encoder=False, eval_metric="logloss"
)

# Fit on training samples that reached SC
model_sign.fit(X_layer2.loc[X_train.index][mask_train_sc], y_train_sign[mask_train_sc])

# Serialize and store both models
model_bytes_sc = pickle.dumps(model_sc)
model_bytes_sign = pickle.dumps(model_sign)

model_entry = pd.DataFrame({
    "TRAIN_DATE": [pd.Timestamp.now()],
    "MODEL_SC": [model_bytes_sc],
    "MODEL_SIGN": [model_bytes_sign]
})

session.write_pandas(model_entry, "ML_CASCADE_MODELS", auto_create_table=True, overwrite=True)
return "✅ Training completed and models stored in ML_CASCADE_MODELS"